__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

4415


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

4415


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
207,2461519928,Участок 6 сот. (ИЖС),3000000,,товарищество собственников недвижимости Лабораторная Балка,"Продам свой участок в перспективном месте, рядом с центром. \nЕсть небольшой уклон, свет подключён, вода городская, круглогодично. Жилого дома нет. \nЗабор в круг старая рабица, нужно поднимать. \nВсе вопросы по телефону. \nАгентам прошу не звонить, сам работа",6,2022-09-08 16:04:40.227,sevastopol,6.00,True,3.00,0.50,4-8
2678,2514588998,Участок 5 сот. (ИЖС),2000000,,"пос. городского типа Кача, ул. Константина Арцеулова","Продается участок под Ижс в поселке Кача. Ул. Константина Арцеулова д.1. Удобное месторасположение: подъезд с 3-х сторон, тихое место, до моря 15 мин. Пешком. Хороший торг при осмотре уместен.",11,2022-09-13 13:56:54.600,sevastopol,5.00,True,2.00,0.40,4-8
80,2279307179,Участок 10 сот. (ИЖС),3000000,,ул. Разведчика Колобуна,"Продажа земельного участка 10 соток — Ижс без Комиссии для Покупателя. \n\nКадастровый номер 91:04:002013:27.\n\nКатегория земель: Земли поселений (земли населенных пунктов). \n\nЕсть возможность разделить участок на 4, 5, 6 или 7 соток за 300 тысяч рублей одна",2,2022-09-08 16:04:40.227,sevastopol,10.00,True,3.00,0.30,8-20


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4415 entries, 0 to 4414
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            4415 non-null   int64         
 1   title               4415 non-null   object        
 2   price               4415 non-null   int64         
 3   obj_name            4415 non-null   object        
 4   adr                 4415 non-null   object        
 5   description         4415 non-null   object        
 6   avito_page          4415 non-null   int64         
 7   ts                  4415 non-null   datetime64[ns]
 8   place               4415 non-null   object        
 9   area                4415 non-null   float64       
 10  is_IJS              4415 non-null   bool          
 11  priceM              4415 non-null   float64       
 12  priceMU             4415 non-null   float64       
 13  area_size_category  4415 non-null   category    

---

In [14]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

4415


### обновляем таблицу адресов

In [15]:
from lib.locator import AddressTransformer

data['adr'] = AddressTransformer().transform( data['adr'] )

places = { # поле 'place' и название города
#'bahchisaray': 'Бахчисарай',
#      'sudak': 'Судак',
 'sevastopol': 'Севастополь,',
}

for k,v in places.items():
    print(k,v) # докидываем в строку адреса название города если его нет
    f =  (data['place']==k) & (~data['adr'].str.match(v))
    data.loc[f,'adr'] = v +' '+ data.loc[f,'adr']

sevastopol Севастополь,


---

In [16]:
from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev
from lib.locator import GeocoderYandex
from yandex_keys import keys


loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[2]),
        # address_transformer=AddressTransformerSev(),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

[INFO    ] 2022-09-15 23:10:42 | LocationUpdater: 9821 addresses in location table
[INFO    ] 2022-09-15 23:10:42 | LocationUpdater: 9821 addresses total
[INFO    ] 2022-09-15 23:10:42 | LocationUpdater: 9821 addresses defined
[INFO    ] 2022-09-15 23:10:42 | LocationUpdater: 0 addresses undefined


In [17]:
loc.sample(3)

,adr,latitude,longitude,truncated
5483,"улица Адмирала Макарова, 15/2",44.59,33.52,False
3150,"Севастополь, Линкорная ул.",44.59,33.57,False
6727,"ТСН СТ Морзаводец, Малиновая ул., 78",44.54,33.57,False


In [18]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [19]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

всего записей: 4426
записей без геометки: 0


In [20]:
assert len(data)>0
data.to_pickle(result_file_path)

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4426 entries, 0 to 4425
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            4426 non-null   int64         
 1   title               4426 non-null   object        
 2   price               4426 non-null   int64         
 3   obj_name            4426 non-null   object        
 4   adr                 4426 non-null   object        
 5   description         4426 non-null   object        
 6   avito_page          4426 non-null   int64         
 7   ts                  4426 non-null   datetime64[ns]
 8   place               4426 non-null   object        
 9   area                4426 non-null   float64       
 10  is_IJS              4426 non-null   bool          
 11  priceM              4426 non-null   float64       
 12  priceMU             4426 non-null   float64       
 13  area_size_category  4426 non-null   category    